In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('HEtrain.csv')
df.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1


In [3]:
df.shape

(3235, 6)

In [4]:
df=df.dropna()

In [5]:
df.shape

(3227, 6)

In [6]:
X=df.drop('sentiment_class',axis=1)
y=df['sentiment_class']

In [7]:
X

,id,original_text,lang,retweet_count,original_author
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794
...,...,...,...,...,...
3230,1.245745e+18,"To all my sisters ,my sisters -in -law and als...",en,0,TheBookTweeters
3231,1.244368e+18,"Happy Mother’s Day to all the Mums, Step Mums,...",en,0,MPWTW
3232,1.245675e+18,Happy Mothers Day to the craziest woman I know...,en,0,localcreativity
3233,1.243976e+18,Happy Mother's Day to my amazing wife! We both...,en,0,LoveluK77651882


In [8]:
import tensorflow as tf

D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [10]:
voc_size=5000

In [11]:
messages=X.copy()
messages['original_text'][1]

"Happy Mothers Day Mum - I'm sorry I can't be there to bring you Mothers day flowers & a cwtch - honestly at this point I'd walk on hot coals to be able to. But I'll be there with bells on as soon as I can be. Love you lots xxx (p.s we need more photos!) https:// photos.app.goo.gl/M3vXBLrsCzD4TE bY7 …"

In [12]:
messages.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\gaurav
[nltk_data]     sahani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
### Dataset Preprocessing
from nltk.stem import WordNetLemmatizer 
WL = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['original_text'][i])
    review = review.lower()
    review = review.split()
    
    review = [WL.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
corpus

['happy mothersday amazing mother know hard able see mother today u protect vulnerable member society beatcoronavirus pic twitter com va nfjfq b',
 'happy mother day mum sorry bring mother day flower cwtch honestly point walk hot coal able bell soon love lot xxx p need photo http photo app goo gl vxblrsczd te',
 'happy mother day mother day work today quiet time reflect dog walk finish jigsaw garden learn guitar chord drunk strawberry gin tonic watch lee even dvd favourite place visit isolate pic twitter com gz xvvf f',
 'happy mother day beautiful woman royalty soothes mummy jeremy emerald prayforroksie ultimateloveng pic twitter com oeeti pvv',
 'remembering amazing lady made late grandmother iris mum carol great grandmother ethel missed never forgotten happy mother day great mum love sent xxxx pic twitter com xzzzdeybje',
 'happy mother day bratz world familiez yasmin mum portia bratz bratzworldfamiliez bratz bratzkidz bratzyasmin bratzportia theonlygirlswithapassionforfashion bratz

In [16]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4905,
  4707,
  2083,
  4051,
  2007,
  33,
  261,
  1053,
  4051,
  3218,
  539,
  4650,
  3278,
  4297,
  4772,
  3239,
  3507,
  401,
  3974,
  1463,
  79,
  1335],
 [4905,
  4051,
  3174,
  3561,
  987,
  1955,
  4051,
  3174,
  3135,
  76,
  4980,
  2965,
  3131,
  1440,
  2102,
  261,
  1914,
  995,
  895,
  3678,
  1098,
  1197,
  1934,
  3863,
  4496,
  3863,
  3410,
  4716,
  4047,
  4292,
  1614],
 [4905,
  4051,
  3174,
  4051,
  3174,
  4871,
  3218,
  4196,
  225,
  1509,
  3789,
  3131,
  1661,
  2126,
  4600,
  1670,
  960,
  3086,
  2467,
  1096,
  4743,
  974,
  3400,
  3023,
  923,
  4570,
  4512,
  1875,
  4805,
  3694,
  3507,
  401,
  3974,
  3115,
  4189,
  3890],
 [4905,
  4051,
  3174,
  2850,
  708,
  2200,
  172,
  2225,
  1192,
  2248,
  1560,
  1464,
  3507,
  401,
  3974,
  1267,
  1467],
 [509,
  2083,
  3691,
  259,
  3069,
  1204,
  4926,
  3561,
  4593,
  4494,
  1204,
  2593,
  666,
  1425,
  750,
  4905,
  4051,
  3174,
  4494,
  3561,
  895,
  3443

In [17]:
sent_length=25
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1463   79 1335]
 [4051 3174 3135 ... 4047 4292 1614]
 [3131 1661 2126 ... 3115 4189 3890]
 ...
 [   0 4905 4051 ...  566 4839 3567]
 [ 895 3526 2604 ... 4432  752   71]
 [   0    0    0 ... 2302 2880 4998]]


In [18]:
embedded_docs[0]

array([   0,    0,    0, 4905, 4707, 2083, 4051, 2007,   33,  261, 1053,
       4051, 3218,  539, 4650, 3278, 4297, 4772, 3239, 3507,  401, 3974,
       1463,   79, 1335])

In [19]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
len(embedded_docs),y.shape

(3227, (3227,))

In [21]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [22]:
y_final

array([ 0,  0, -1, ...,  0,  0, -1], dtype=int64)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=22)

In [24]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Train on 2162 samples, validate on 1065 samples
Epoch 1/20
2162/2162 [==============================] - 1s 663us/sample - loss: 0.3279 - acc: 0.5296 - val_loss: -0.1077 - val_acc: 0.5089
Epoch 2/20
2162/2162 [==============================] - 1s 457us/sample - loss: 0.0320 - acc: 0.5324 - val_loss: -0.0896 - val_acc: 0.5089
Epoch 3/20
2162/2162 [==============================] - 1s 449us/sample - loss: -0.3275 - acc: 0.5370 - val_loss: -0.0142 - val_acc: 0.4808
Epoch 4/20
2162/2162 [==============================] - 1s 419us/sample - loss: -2.2906 - acc: 0.6166 - val_loss: -0.1522 - val_acc: 0.4986
Epoch 5/20
2162/2162 [==============================] - 1s 401us/sample - loss: -3.1750 - acc: 0.6277 - val_loss: -0.0602 - val_acc: 0.4554
Epoch 6/20
2162/2162 [==============================] - 1s 401us/sample - loss: -4.7290 - acc: 0.6425 - val_loss: -0.6381 - val_acc: 0.4864
Epoch 7/20
2162/2162 [==============================] - 1s 387us/sample - loss: -5.5358 - acc: 0.6804 - val_loss: 

In [25]:
y_pred=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[  0, 206,  66],
       [  0, 429, 113],
       [  0, 179,  72]], dtype=int64)

In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.4704225352112676